In [1]:
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, RobustScaler, Normalizer, MinMaxScaler, MaxAbsScaler, scale, Normalizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor as KNR
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import kurtosis, skew
from string import ascii_letters
from functools import partial
import warnings
warnings.filterwarnings("ignore")
import os
os.getcwd()
sns.set(style="darkgrid")

In [2]:
! pip install featuretools

In [3]:
import featuretools as ft
from io import BytesIO as BI
%matplotlib inline
# import all data
# first X rows are taken for faster calculations, substitute this by whole dataset

bureau = pd.read_csv('OneHot-bureau.csv')

app_joint = pd.read_csv('OneHot-application_joint.csv')

bureau_balance = pd.read_csv('OneHot-bureau_balance.csv')

cre_balance = pd.read_csv('OneHot-credit_card_payments.csv')

ins_payments = pd.read_csv('OneHot-installments_payments.csv')

pre_application = pd.read_csv('OneHot-previous_application.csv')

POS_balance = pd.read_csv('OneHot-POS_CASH_balance.csv')

## Feature Engineering

In [4]:
previous = pre_application.copy()
cash = POS_balance.copy()
installments = ins_payments.copy()
credit = cre_balance.copy()

### We can draw samples based on different training windows. For example, 1 month, 3 months, 6 months, 1 year, 3 years, 100 years

### Train the bureau_matrix

#### Bureau

In [5]:
# initialize entityset
es = ft.EntitySet('bureau_data')

today = pd.to_datetime('2018-06-11')
bureau['DAYS_CREDIT'] = today + pd.to_timedelta(bureau['DAYS_CREDIT'], unit='d')

# add entities (brueau table)
es = es.entity_from_dataframe(
    entity_id = 'bureau', 
    dataframe = bureau,
    index = 'SK_ID_BUREAU',
    time_index = 'DAYS_CREDIT',
)

es

Entityset: bureau_data
  Entities:
    bureau [Rows: 3022, Columns: 31]
  Relationships:
    No relationships

#### Bureau_balance

In [6]:
# if error occurs, try: today = pd.to_datetime('2018-06')
bureau_balance['MONTHS_BALANCE'] = today + pd.to_timedelta(bureau_balance['MONTHS_BALANCE'], unit='M')

# Entities that do not have a unique index
es = es.entity_from_dataframe(
    entity_id = 'bureau_balance', 
    dataframe = bureau_balance,
    make_index = True,
    index = 'bureaubalance_index',
    time_index = 'MONTHS_BALANCE'
)

r_bureau_balance = ft.Relationship(es['bureau']['SK_ID_BUREAU'], es['bureau_balance']['SK_ID_BUREAU'])
es = es.add_relationships([r_bureau_balance])
es

Entityset: bureau_data
  Entities:
    bureau_balance [Rows: 73514, Columns: 12]
    bureau [Rows: 3022, Columns: 31]
  Relationships:
    bureau_balance.SK_ID_BUREAU -> bureau.SK_ID_BUREAU

In [7]:
%%time
# define cut-off times
# cut-off times are the "right" time values to be used for feature calculation without future leaks
# none in our case

cutoff_times = pd.DataFrame(bureau.SK_ID_BUREAU)
cutoff_times['time'] = today

# add last_time_index
es.add_last_time_indexes()

CPU times: user 858 ms, sys: 14 ms, total: 872 ms
Wall time: 862 ms


In [8]:
def merge_matrix(fm_sum_, fm_, period, parent_cols):
  child_cols = fm_.drop(columns = parent_cols).columns
  fm_ = fm_[child_cols].add_prefix(str(period) + " MONTHS(").add_suffix(")")
  return fm_sum_.join(fm_)
  
def period_matrix(period, es = es, target_entity = 'prev_apps'):
  # Specify the fm with the first period
  fm_sum, feature_names = ft.dfs(entityset = es, target_entity = target_entity,
                                 drop_contains = ['SK_ID_CURR','SK_ID_PREV','SK_ID_BUREAU'],
                                 agg_primitives=[
                                   'mean',
                                   'count',
                                   'last',
                                   'num_unique'
                                 ], 
                                 trans_primitives=[
                                   'time_since_previous',
                                   'weekday',
                                   'month',
                                   'year'
                                 ],
                                 max_depth=2,
                                 cutoff_time=cutoff_times,
                                 training_window=ft.Timedelta(period[0]*31, "d"), # use only last X days in computations
                                 max_features=3000,
                                 chunk_size=10000,
                                 verbose=True
                                )
  parent_cols = [col for col in fm_sum.columns 
                 if (('(cash.' not in col) & ('(installments.' not in col) & ('(credit.' not in col)
                     & ('(bureau_balance.' not in col))]
  fm_sum = merge_matrix(fm_sum[parent_cols], fm_sum, period[0], parent_cols)
  
  for i in period[1:]:
    # DFS with specified primitives
    fm, feature_names = ft.dfs(entityset = es, target_entity = target_entity,
                           drop_contains = ['SK_ID_CURR','SK_ID_PREV','SK_ID_BUREAU'],
                           agg_primitives=[
                             'mean',
                             'count',
                             'last',
                             'num_unique'
                           ], 
                           trans_primitives=[
                             'time_since_previous',
                             'weekday',
                             'month',
                             'year'
                           ],
                           max_depth=2,
                           cutoff_time=cutoff_times,
                           training_window=ft.Timedelta(31*i, "d"), # use only last X days in computations
                           max_features=3000,
                           chunk_size=10000,
                           verbose=True
                          )
    fm_sum = merge_matrix(fm_sum, fm, i, parent_cols)
    print('%d Total Features' % len(feature_names))
  
  # Merge with the whole training time matrix
  fm, feature_names = ft.dfs(entityset = es, target_entity = target_entity,
                       drop_contains = ['SK_ID_CURR','SK_ID_PREV','SK_ID_BUREAU'],
                       agg_primitives=[
                         'mean',
                         'count',
                         'last',
                         'num_unique'
                       ], 
                       trans_primitives=[
                         'time_since_previous',
                         'weekday',
                         'month',
                         'year'
                       ],
                       max_depth=2,
                       cutoff_time=cutoff_times,
                       #training_window=ft.Timedelta(31*i, "d"), # use only last X days in computations
                       max_features=3000,
                       chunk_size=10000,
                       verbose=True
                      )
  fm = fm.drop(columns = parent_cols)
  fm_sum = fm_sum.join(fm)
  return fm_sum, fm_sum.columns.values.tolist()

In [9]:
fm, features_names = period_matrix(period = [6, 36], es = es, target_entity = 'bureau')
#features_names

Built 59 features
Elapsed: 00:00 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 1/1 chunks
Built 59 features
Elapsed: 00:01 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 1/1 chunks
59 Total Features
Built 59 features
Elapsed: 00:01 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 1/1 chunks


In [10]:
drop_list = [col for col in fm.columns if '(LAST(' in col]
fm = fm.drop(columns = drop_list)

In [11]:
# merge the generated feature_matrix to the original dataframe
bureau = bureau.merge(fm.reset_index())

### Train the previous_matrix

#### Previous

In [12]:
today = pd.to_datetime('2018-06-11')
previous['DAYS_DECISION'] = today + pd.to_timedelta(previous['DAYS_DECISION'], unit='d')

In [13]:
# initialize entityset
es = ft.EntitySet('previous_data')

# add entities (previous applications table)
es = es.entity_from_dataframe(
    entity_id = 'prev_apps', 
    dataframe = previous,
    index = 'SK_ID_PREV',
    time_index = 'DAYS_DECISION'
)

#### Installments

In [14]:
# Create new feature for installments
installments['DAYS_DELAYS'] = installments['DAYS_ENTRY_PAYMENT'] - installments['DAYS_INSTALMENT']

# if error occurs, try: today = pd.to_datetime('2018-06')
installments['DAYS_INSTALMENT'] = today + pd.to_timedelta(installments['DAYS_INSTALMENT'], unit='d')

In [15]:
# add entities (installments table)
es = es.entity_from_dataframe(
    entity_id = 'installments', 
    dataframe = installments,
    make_index = True,
    time_index = 'DAYS_INSTALMENT',
    index = 'installment_index'
)

r_installments = ft.Relationship(es['prev_apps']['SK_ID_PREV'], es['installments']['SK_ID_PREV'])
es = es.add_relationship(r_installments)
es

Entityset: previous_data
  Entities:
    installments [Rows: 22751, Columns: 20]
    prev_apps [Rows: 2989, Columns: 148]
  Relationships:
    installments.SK_ID_PREV -> prev_apps.SK_ID_PREV

#### Credit_card

In [16]:
today = pd.to_datetime('2018-06-11')
credit['MONTHS_BALANCE'] = today + pd.to_timedelta(credit['MONTHS_BALANCE'], unit='M')

# add entities (credit_card_balance table)
es = es.entity_from_dataframe(
    entity_id = 'credit', 
    dataframe = credit,
    make_index = True,
    index = 'credit_index',
    time_index = 'MONTHS_BALANCE'
)

r_previous_credit = ft.Relationship(es['prev_apps']['SK_ID_PREV'], es['credit']['SK_ID_PREV'])
es = es.add_relationship(r_previous_credit)
es

Entityset: previous_data
  Entities:
    credit [Rows: 4922, Columns: 27]
    installments [Rows: 22751, Columns: 20]
    prev_apps [Rows: 2989, Columns: 148]
  Relationships:
    installments.SK_ID_PREV -> prev_apps.SK_ID_PREV
    credit.SK_ID_PREV -> prev_apps.SK_ID_PREV

#### POS_cash

In [17]:
# change the time variable
today = pd.to_datetime('2018-06-11')
cash['MONTHS_BALANCE'] = today + pd.to_timedelta(cash['MONTHS_BALANCE'], unit='M')

In [18]:
# add entities (previous applications table)
es = es.entity_from_dataframe(
    entity_id = 'cash', 
    dataframe = cash,
    make_index = True,
    index = 'cash_index',
    time_index = 'MONTHS_BALANCE'
)

r_cash = ft.Relationship(es['prev_apps']['SK_ID_PREV'], es['cash']['SK_ID_PREV'])
es = es.add_relationship(r_cash)
es

Entityset: previous_data
  Entities:
    credit [Rows: 4922, Columns: 27]
    cash [Rows: 17821, Columns: 14]
    installments [Rows: 22751, Columns: 20]
    prev_apps [Rows: 2989, Columns: 148]
  Relationships:
    installments.SK_ID_PREV -> prev_apps.SK_ID_PREV
    credit.SK_ID_PREV -> prev_apps.SK_ID_PREV
    cash.SK_ID_PREV -> prev_apps.SK_ID_PREV

In [19]:
%%time
# define cut-off times
# cut-off times are the "right" time values to be used for feature calculation without future leaks
# none in our case

cutoff_times = pd.DataFrame(previous.SK_ID_PREV)
cutoff_times['time'] = today

# add last_time_index
es.add_last_time_indexes()

CPU times: user 2.52 s, sys: 10.7 ms, total: 2.53 s
Wall time: 2.52 s


In [20]:
fm, features_names = period_matrix(period = [1, 3], es = es, target_entity = 'prev_apps')
#features_names

Built 276 features
Elapsed: 00:01 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 1/1 chunks
Built 276 features
Elapsed: 00:01 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 1/1 chunks
276 Total Features
Built 276 features
Elapsed: 00:02 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 1/1 chunks


In [21]:
drop_list = [col for col in fm.columns if '(LAST(' in col]
fm = fm.drop(columns = drop_list)

In [22]:
# merge the generated feature_matrix to the original dataframe
previous = previous.merge(fm.reset_index())

### Train the Main Applications

#### Main Applications

In [23]:
# Hand Crafted Features
app_joint['annuity_income_percentage'] = app_joint['AMT_ANNUITY'] / app_joint['AMT_INCOME_TOTAL']
app_joint['car_to_birth_ratio'] = app_joint['OWN_CAR_AGE'] / app_joint['DAYS_BIRTH']
app_joint['car_to_employ_ratio'] = app_joint['OWN_CAR_AGE'] / app_joint['DAYS_EMPLOYED']
app_joint['children_ratio'] = app_joint['CNT_CHILDREN'] / app_joint['CNT_FAM_MEMBERS']
app_joint['credit_to_annuity_ratio'] = app_joint['AMT_CREDIT'] / app_joint['AMT_ANNUITY']
app_joint['credit_to_goods_ratio'] = app_joint['AMT_CREDIT'] / app_joint['AMT_GOODS_PRICE']
app_joint['credit_to_income_ratio'] = app_joint['AMT_CREDIT'] / app_joint['AMT_INCOME_TOTAL']
app_joint['days_employed_percentage'] = app_joint['DAYS_EMPLOYED'] / app_joint['DAYS_BIRTH']
app_joint['income_credit_percentage'] = app_joint['AMT_INCOME_TOTAL'] / app_joint['AMT_CREDIT']
app_joint['income_per_child'] = app_joint['AMT_INCOME_TOTAL'] / (1 + app_joint['CNT_CHILDREN'])
app_joint['income_per_person'] = app_joint['AMT_INCOME_TOTAL'] / app_joint['CNT_FAM_MEMBERS']
app_joint['payment_rate'] = app_joint['AMT_ANNUITY'] / app_joint['AMT_CREDIT']
app_joint['phone_to_birth_ratio'] = app_joint['DAYS_LAST_PHONE_CHANGE'] / app_joint['DAYS_BIRTH']
app_joint['phone_to_employ_ratio'] = app_joint['DAYS_LAST_PHONE_CHANGE'] / app_joint['DAYS_EMPLOYED']

# Family members
app_joint['cnt_non_child'] = app_joint['CNT_FAM_MEMBERS'] - app_joint['CNT_CHILDREN']
app_joint['child_to_non_child_ratio'] = app_joint['CNT_CHILDREN'] / app_joint['cnt_non_child']
app_joint['income_per_non_child'] = app_joint['AMT_INCOME_TOTAL'] / app_joint['cnt_non_child']
app_joint['credit_per_person'] = app_joint['AMT_CREDIT'] / app_joint['CNT_FAM_MEMBERS']
app_joint['credit_per_child'] = app_joint['AMT_CREDIT'] / (1 + app_joint['CNT_CHILDREN'])
app_joint['credit_per_non_child'] = app_joint['AMT_CREDIT'] / app_joint['cnt_non_child']

In [24]:
# initialize entityset
es = ft.EntitySet('application_data')

# add entities (application table itself)
es.entity_from_dataframe(
    entity_id='apps', # define entity id
    dataframe=app_joint.drop('TARGET', axis=1), # select underlying data
    index='SK_ID_CURR', # define unique index column
    # specify some datatypes manually (if needed)
    variable_types={
        f: ft.variable_types.Categorical 
        for f in app_joint.columns if f.startswith('FLAG_')
    }
)

Entityset: application_data
  Entities:
    apps [Rows: 600, Columns: 141]
  Relationships:
    No relationships

#### Previous application

In [25]:
# add entities (previous applications table)
es = es.entity_from_dataframe(
    entity_id = 'prev_apps', 
    dataframe = previous,
    index = 'SK_ID_PREV',
    time_index = 'DAYS_DECISION',
)

# add relationships
r_app_cur_to_app_prev = ft.Relationship(
    es['apps']['SK_ID_CURR'],
    es['prev_apps']['SK_ID_CURR']
)

# Add the relationship to the entity set
es = es.add_relationship(r_app_cur_to_app_prev)

# check constructed entity set
es

Entityset: application_data
  Entities:
    apps [Rows: 600, Columns: 141]
    prev_apps [Rows: 410, Columns: 386]
  Relationships:
    prev_apps.SK_ID_CURR -> apps.SK_ID_CURR

#### Bureau

In [26]:
# add entities (brueau table)
es = es.entity_from_dataframe(
    entity_id = 'bureau', 
    dataframe = bureau,
    index = 'SK_ID_BUREAU',
    time_index = 'DAYS_CREDIT'
)

# add relationships
r_app_cur_to_bureau = ft.Relationship(
    es['apps']['SK_ID_CURR'],
    es['bureau']['SK_ID_CURR']
)

# Add the relationship to the entity set
es = es.add_relationship(r_app_cur_to_bureau)
es

Entityset: application_data
  Entities:
    bureau [Rows: 2064, Columns: 83]
    apps [Rows: 600, Columns: 141]
    prev_apps [Rows: 410, Columns: 386]
  Relationships:
    prev_apps.SK_ID_CURR -> apps.SK_ID_CURR
    bureau.SK_ID_CURR -> apps.SK_ID_CURR

In [27]:
%%time
# define cut-off times
# cut-off times are the "right" time values to be used for feature calculation without future leaks
# none in our case

cutoff_times = pd.DataFrame(app_joint.SK_ID_CURR)
cutoff_times['time'] = today

# add last_time_index
es.add_last_time_indexes()

CPU times: user 395 ms, sys: 20.1 ms, total: 415 ms
Wall time: 381 ms


In [28]:
def merge_matrix(fm_sum_, fm_, period, parent_cols):
  child_cols = fm_.drop(columns = parent_cols).columns
  fm_ = fm_[child_cols].add_prefix(str(period) + " MONTHS(").add_suffix(")")
  return fm_sum_.join(fm_)
  
def period_matrix(period, es = es, target_entity = 'prev_apps'):
  # Specify the fm with the first period
  fm_sum, feature_names = ft.dfs(entityset = es, target_entity = target_entity,
                                 drop_contains = ['SK_ID_CURR','SK_ID_PREV','SK_ID_BUREAU'],
                                 agg_primitives=[
                                   'mean',
                                   'std',
                                   'median',
                                   'min',
                                   'max',
                                   'count',
                                   'last',
                                   'num_unique'
                                 ], 
                                 trans_primitives=[
                                   'time_since_previous',
                                   'weekday',
                                   'month',
                                   'year'
                                 ],
                                 max_depth=2,
                                 cutoff_time=cutoff_times,
                                 training_window=ft.Timedelta(period[0]*31, "d"), # use only last X days in computations
                                 max_features=3000,
                                 chunk_size=10000,
                                 verbose=True
                                )
  parent_cols = [col for col in fm_sum.columns 
                 if (('(cash.' not in col) & ('(installments.' not in col) & ('(credit.' not in col)
                     & ('(bureau_balance.' not in col) & ('(bureau.' not in col) & ('(prev_apps.' not in col))]
  fm_sum = merge_matrix(fm_sum[parent_cols], fm_sum, period[0], parent_cols)
  
  for i in period[1:]:
    # DFS with specified primitives
    fm, feature_names = ft.dfs(entityset = es, target_entity = target_entity,
                           drop_contains = ['SK_ID_CURR','SK_ID_PREV','SK_ID_BUREAU'],
                           agg_primitives=[
                             'mean',
                             'std',
                             'median',
                             'min',
                             'max',
                             'count',
                             'last',
                             'num_unique'
                           ], 
                           trans_primitives=[
                             'time_since_previous',
                             'weekday',
                             'month',
                             'year'
                           ],
                           max_depth=2,
                           cutoff_time=cutoff_times,
                           training_window=ft.Timedelta(31*i, "d"), # use only last X days in computations
                           max_features=3000,
                           chunk_size=10000,
                           verbose=True
                          )
    fm_sum = merge_matrix(fm_sum, fm, i, parent_cols)
    print('%d Total Features' % len(feature_names))
  
  # Merge with the whole training time matrix
  fm, feature_names = ft.dfs(entityset = es, target_entity = target_entity,
                       drop_contains = ['SK_ID_CURR','SK_ID_PREV','SK_ID_BUREAU'],
                       agg_primitives=[
                         'mean',
                         'std',
                         'median',
                         'min',
                         'max',
                         'count',
                         'last',
                         'num_unique'
                       ], 
                       trans_primitives=[
                         'time_since_previous',
                         'weekday',
                         'month',
                         'year'
                       ],
                       max_depth=2,
                       cutoff_time=cutoff_times,
                       #training_window=ft.Timedelta(31*i, "d"), # use only last X days in computations
                       max_features=3000,
                       chunk_size=10000,
                       verbose=True
                      )
  fm = fm.drop(columns = parent_cols)
  fm_sum = fm_sum.join(fm)
  return fm_sum, fm_sum.columns.values.tolist()

In [29]:
fm, features_names = period_matrix(period = [6, 36], es = es, target_entity = 'apps')
#features_names

Built 2926 features
Elapsed: 00:15 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 1/1 chunks
Built 2926 features
Elapsed: 00:35 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 1/1 chunks
2926 Total Features
Built 2926 features
Elapsed: 00:36 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 1/1 chunks


In [30]:
drop_list = [col for col in fm.columns if '(last(' in col]
fm = fm.drop(columns = drop_list)

In [31]:
# merge the generated feature_matrix to the original dataframe
app_joint = app_joint.merge(fm.reset_index())

In [32]:
app_joint = app_joint.loc[:,app_joint.columns != 'TARGET'].fillna(0).join(app_joint['TARGET'])

In [ ]:
app_joint.to_csv('Feature_Matrix_app_joint.csv', encoding='utf-8', index=False)